In [1]:
import pandas as pd

train_prob = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/데이콘_lg/train_problem_data.csv', thousands = ',')
train_qual = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/데이콘_lg/train_quality_data.csv', thousands = ',')
train_err = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/데이콘_lg/train_err_data.csv', thousands = ',')

In [2]:
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

In [3]:
train_qual

time  user_id       fwver  ...  quality_10  quality_11  quality_12
0       20201129090000    10000  05.15.2138  ...           4           0           0
1       20201129090000    10000  05.15.2138  ...           4           0           0
2       20201129090000    10000  05.15.2138  ...           4           0           0
3       20201129090000    10000  05.15.2138  ...           4           0           0
4       20201129090000    10000  05.15.2138  ...           4           0           0
...                ...      ...         ...  ...         ...         ...         ...
828619  20201124033000    24997  04.22.1778  ...          17           0           0
828620  20201124033000    24997  04.22.1778  ...          17           0           0
828621  20201124033000    24997  04.22.1778  ...          17           0           0
828622  20201124033000    24997  04.22.1778  ...          17           0           0
828623  20201124033000    24997  04.22.1778  ...          17           0           0

[828624 rows x 16 columns]

In [4]:
import datetime as dt

# 필요한 함수 정의
def make_datetime(x):
    # string 타입의 Time column을 datetime 타입으로 변경
    x     = str(x)
    year  = int(x[:4])
    month = int(x[4:6])
    day   = int(x[6:8])
    hour  = int(x[8:10])
    #min  = int(x[10:12])
    #sec  = int(x[12:])
    return dt.datetime(year, month, day, hour)

In [5]:
train_qual['datetime'] = train_qual['time'].apply(make_datetime)

train_qual['year'] = train_qual.loc[:,'datetime'].dt.year
train_qual['month'] = train_qual.loc[:,'datetime'].dt.month

min_day = train_qual.datetime.min().date()

train_qual['days'] = train_qual.loc[:,'datetime'].dt.date - min_day
train_qual['days'] = train_qual['days'].dt.days

train_qual['hour'] = train_qual.loc[:, 'datetime'].dt.hour

In [6]:
train_qual.head()

time  user_id       fwver  quality_0  ...  year  month  days  hour
0  20201129090000    10000  05.15.2138        0.0  ...  2020     11    29     9
1  20201129090000    10000  05.15.2138        0.0  ...  2020     11    29     9
2  20201129090000    10000  05.15.2138        0.0  ...  2020     11    29     9
3  20201129090000    10000  05.15.2138        0.0  ...  2020     11    29     9
4  20201129090000    10000  05.15.2138        0.0  ...  2020     11    29     9

[5 rows x 21 columns]

In [7]:
train_qual.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 828624 entries, 0 to 828623
Data columns (total 21 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   time        828624 non-null  int64         
 1   user_id     828624 non-null  int64         
 2   fwver       788544 non-null  object        
 3   quality_0   684192 non-null  float64       
 4   quality_1   828624 non-null  int64         
 5   quality_2   788511 non-null  float64       
 6   quality_3   828624 non-null  int64         
 7   quality_4   828624 non-null  int64         
 8   quality_5   828604 non-null  float64       
 9   quality_6   828624 non-null  int64         
 10  quality_7   828624 non-null  int64         
 11  quality_8   828624 non-null  int64         
 12  quality_9   828624 non-null  int64         
 13  quality_10  828624 non-null  int64         
 14  quality_11  828624 non-null  int64         
 15  quality_12  828624 non-null  int64         
 16  da

In [8]:
train_qual.drop(columns = ['time', 'datetime'], inplace= True)

In [15]:
train_qual.dropna(axis = 0, inplace=True)

In [16]:
# qual column간 연관관계
X = train_qual.drop(columns = 'user_id')
y = train_qual[['user_id']]

In [17]:
X['fwver'] = X['fwver'].str.replace('.', '').astype('int64')

In [18]:
X

fwver  quality_0  quality_1  quality_2  ...  year  month  days  hour
0       5152138        0.0          0        0.0  ...  2020     11    29     9
1       5152138        0.0          0        0.0  ...  2020     11    29     9
2       5152138        0.0          0        0.0  ...  2020     11    29     9
3       5152138        0.0          0        0.0  ...  2020     11    29     9
4       5152138        0.0          0        0.0  ...  2020     11    29     9
...         ...        ...        ...        ...  ...   ...    ...   ...   ...
828619  4221778        0.0          0        0.0  ...  2020     11    24     3
828620  4221778        0.0          0        0.0  ...  2020     11    24     3
828621  4221778        0.0          0        0.0  ...  2020     11    24     3
828622  4221778        0.0          0        0.0  ...  2020     11    24     3
828623  4221778        0.0          0        0.0  ...  2020     11    24     3

[684139 rows x 18 columns]

In [ ]:
from lightgbm import plot_importance
import matplotlib.pyplot as plt
%matplotlib inline

lgbm = LGBMClassifier()
lgbm.fit(X, y)

fig, ax = plt.subplots(figsize=(10, 12))
plot_importance(lgbm, ax=ax)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
# from xgboost import plot_importance
# import matplotlib.pyplot as plt
# %matplotlib inline

# xgb = XGBClassifier()
# xgb.fit(X, y)

# fig, ax = plt.subplots(figsize=(10, 12))
# plot_importance(xgb, ax=ax)